<a href="https://colab.research.google.com/github/itssofiarce/AA2-TP2-Alomar-Arce/blob/main/ej_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Descripción:
En este problema, se presenta un conjunto de datos que contiene clips de audio correspondientes a oraciones habladas en distintos idiomas.

Dataset:
https://www.tensorflow.org/datasets/catalog/xtreme_s


El dataset proporcionado incluye diversos sub-datasets correspondientes a oraciones habladas en distintos idiomas. Utilizaremos un subconjunto de estos datasets para entrenar un clasificador de idiomas hablados.

Objetivo:
Utilizando el dataset proporcionado, el objetivo es construir un modelo de clasificación utilizando redes neuronales que pueda inferir el idioma correspondiente.

Se solicita entrenar dos modelos de distintas arquitecturas y comparar los resultados:

Modelo convolucional sobre los espectrogramas de los clips.
Modelo recurrente sobre los espectrogramas de los clips.

Ver https://colab.research.google.com/github/FCEIA-AAII/lab11/blob/master/lab11-a.ipynb como ejemplo de obtención de espectrogramas a partir de clips de audio.

# Librerías

In [1]:
rm -rf data

In [3]:
!pip install tensorflow

In [4]:
!pip install pydub

In [5]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

import tensorflow as tf
import tensorflow_datasets as tfds #  La versión estable, lanzada cada pocos meses.
import pandas as pd
from typing import Optional
from tensorflow_datasets.core import dataset_info # de aca viene el dataset

# Construcción y carga del Dataset


In [6]:
tfds.builder('xtreme_s').info # full_name

tfds.core.DatasetInfo(
    name='xtreme_s',
    full_name='xtreme_s/fleurs.af_za/2.0.0',
    description="""
    FLEURS is the speech version of the FLORES machine translation benchmark, covering 2000 n-way parallel sentences in n=102 languages.
    XTREME-S covers four task families: speech recognition, classification, speech-to-text translation and retrieval. Covering 102
    languages from 10+ language families, 3 different domains and 4
    task families, XTREME-S aims to simplify multilingual speech
    representation evaluation, as well as catalyze research in “universal” speech representation learning.
    
    In this version, only the FLEURS dataset is provided, which covers speech
    recognition and speech-to-text translation.
    """,
    config_description="""
    FLEURS is the speech version of the FLORES machine translation benchmark, covering 2000 n-way parallel sentences in n=102 languages.
    """,
    homepage='https://arxiv.org/abs/2205.12446',
    data_dir=PosixGPa

In [7]:
DATASET_PATH = 'data'

data_dir = pathlib.Path(DATASET_PATH)
lan = {'español':'es_419', 'ingles': 'en_us', 'frances':'fr_fr', 'japones': 'ja_jp'}

In [9]:
for key, value in lan.items():
    # Chequeo que la carpeta de los datasets no esté vacía

    # Traigo solamente una parte del subset de test para agilizar la carga
    tfds.load(f'xtreme_s/fleurs.{value}', split='test[:1%]', shuffle_files=False, data_dir=data_dir / key)

    # Limpiar y quedarme unicamente con los audios que me insteresan

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/2796 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/266 [00:00<?, ? examples/s]

Shuffling data/japones/xtreme_s/fleurs.ja_jp/incomplete.17GOZJ_2.0.0/xtreme_s-validation.tfrecord*...:   0%|  …

Generating test examples...:   0%|          | 0/650 [00:00<?, ? examples/s]

Shuffling data/japones/xtreme_s/fleurs.ja_jp/incomplete.17GOZJ_2.0.0/xtreme_s-test.tfrecord*...:   0%|        …

Dataset xtreme_s downloaded and prepared to data/japones/xtreme_s/fleurs.ja_jp/2.0.0. Subsequent calls will reuse this data.


In [20]:
!rm -rf data/japones/downloads
!rm -rf data/japones/xtreme_s


In [ ]:
tf.data.experimental.save(
    ds, tf_data_path, compression='GZIP'
)

In [21]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=64,
    validation_split=0.2,
    seed=0,
    output_sequence_length=16000,
    subset='both')

label_names = lan.keys()
print()
print("label names:", label_names)

Found 11913 files belonging to 4 classes.
Using 9531 files for training.
Using 2382 files for validation.

label names: dict_keys(['español', 'ingles', 'frances', 'japones'])


# Analisis Exploratorio


In [22]:
train_ds.element_spec

(TensorSpec(shape=(None, 16000, None), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

# Modelo Convolucional

# Modelo Recurrente